In [8]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [4]:
import pandas as pd 
file_path = '/home/jupyter-user5/Camda24_resistance/DataSets/ResistanceCiprofloxacinLoose.tsv.gz'
df = pd.read_csv(file_path, sep='\t', compression='gzip')
df.head()


/opt/conda/envs/TDA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,accession,genus,species,phenotype,mic,3000535,3005091,3000833,3003665,3007433,...,3003285-S531G,3007051-I572F,3007051-I837V,3003294-D105E,3003394-S66P,3005106-A352E,3003937-N514H,3003937-L546V,3003304-E540V,3003304-E504V
0,SRR3138666,Campylobacter,jejuni,Susceptible,0.12,19.0,10.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,SRR3138667,Campylobacter,jejuni,Susceptible,0.06,19.0,6.0,7.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SRR3138668,Campylobacter,jejuni,Susceptible,0.06,16.0,7.0,5.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,SRR3138669,Campylobacter,jejuni,Susceptible,0.06,16.0,7.0,5.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,SRR3138670,Campylobacter,jejuni,Susceptible,0.06,20.0,8.0,6.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Definir características numéricas y categóricas (ajusta estos nombres a las columnas del nuevo archivo)
numerical_features = ['mic'] + [col for col in df.columns if col.startswith('300')]
categorical_features = ['genus', 'species']



In [9]:
# Crear el preprocesador para imputación y estandarización de características numéricas, y codificación One-Hot para categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean')),  # Imputar con la media
            ('scaler', StandardScaler())  # Estandarización
        ]), numerical_features),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar con la moda
            ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Codificación One-Hot
        ]), categorical_features)
    ])

In [10]:
from sklearn.preprocessing import LabelEncoder

# Imputación de etiquetas (phenotype) si faltan valores
y = df['phenotype']
missing_labels = y.isnull().sum()

if missing_labels > 0:
    label_imputer = SimpleImputer(strategy='most_frequent')
    y_imputed = label_imputer.fit_transform(y.values.reshape(-1, 1)).ravel()
else:
    y_imputed = y.values

# Codificación de etiquetas en formato numérico
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_imputed)


In [11]:
# Crear el pipeline final que incluye el preprocesamiento de características
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Aplicar el pipeline a los datos (excluyendo phenotype y accession)
X = df.drop(columns=['phenotype', 'accession'])
X_processed = pipeline.fit_transform(X)

# Mostrar las dimensiones finales de los datos transformados
print(X_processed.shape)


(3881, 5574)


In [12]:
# Aplicar el pipeline a los datos procesados
X_processed = pipeline.fit_transform(X)

# Ver las primeras filas de las etiquetas codificadas
print(y_encoded[:5])


[1 1 1 1 1]


In [13]:
####################################

In [14]:
# Contar los valores faltantes por cada columna
missing_values = df.isnull().sum()

# Mostrar sólo las columnas con valores faltantes
missing_values = missing_values[missing_values > 0]
print(missing_values)


phenotype    1063
mic          1063
dtype: int64


In [15]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Separar las características (X) y las etiquetas (y)
X = df.drop(columns=['phenotype', 'accession'])  # Excluir las etiquetas y columnas irrelevantes
y = df['phenotype']  # Etiquetas

# Definir las características numéricas y categóricas
numerical_features = ['mic'] + [col for col in df.columns if col.startswith('300')]
categorical_features = ['genus', 'species']

# Crear el pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        # Imputar 'mic' con la media
        ('num', SimpleImputer(strategy='mean'), numerical_features),
        
        # Imputar categóricas con la moda y hacer OneHotEncoding
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_features)
    ])

# Imputación de la etiqueta 'phenotype' con la clase más frecuente
label_imputer = SimpleImputer(strategy='most_frequent')

# Aplicar imputación a las etiquetas (y)
y_imputed = label_imputer.fit_transform(y.values.reshape(-1, 1)).ravel()

# Codificar las etiquetas imputadas en formato numérico si es necesario
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_imputed)

# Aplicar el pipeline a las características (X)
X_processed = preprocessor.fit_transform(X)

# Mostrar la forma de los datos procesados
print(X_processed.shape)


(3881, 5574)


In [16]:
# Validar que los valores faltantes fueron correctamente imputados
import numpy as np

print(f"Valores faltantes en X procesado: {np.isnan(X_processed).sum()}")
print(f"Valores faltantes en las etiquetas imputadas: {np.isnan(y_encoded).sum()}")


Valores faltantes en X procesado: 0
Valores faltantes en las etiquetas imputadas: 0


In [17]:
#2

In [18]:
# Verificar los tipos de datos para identificar columnas categóricas
categorical_columns = df.select_dtypes(include=['object']).columns
print("Características categóricas identificadas:", categorical_columns)


Características categóricas identificadas: Index(['accession', 'genus', 'species', 'phenotype'], dtype='object')


In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Características categóricas identificadas
categorical_features = ['genus', 'species']  # Excluimos 'accession' y 'phenotype'

# Crear el pipeline de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        # Imputar y escalar las características numéricas
        ('num', SimpleImputer(strategy='mean'), numerical_features),
        
        # Codificación One-Hot para las características categóricas 'genus' y 'species'
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Crear el pipeline final
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

# Aplicar el pipeline a las características (X) excluyendo 'accession' y 'phenotype'
X = df.drop(columns=['phenotype', 'accession'])
X_processed = pipeline.fit_transform(X)

# Mostrar las dimensiones de los datos transformados
print(X_processed.shape)


(3881, 5574)


In [20]:
# Mostrar las primeras filas del conjunto procesado
import pandas as pd
print(pd.DataFrame(X_processed).head())


   0     1     2     3     4     5     6     7     8     9     ...  5564  \
0  0.12  19.0  10.0   4.0   3.0   3.0   6.0   1.0   1.0   9.0  ...   0.0   
1  0.06  19.0   6.0   7.0   4.0   4.0   4.0   1.0   1.0  13.0  ...   0.0   
2  0.06  16.0   7.0   5.0   3.0   3.0   5.0   2.0   0.0   6.0  ...   0.0   
3  0.06  16.0   7.0   5.0   3.0   3.0   5.0   1.0   0.0   6.0  ...   0.0   
4  0.06  20.0   8.0   6.0   4.0   4.0   5.0   2.0   1.0   9.0  ...   0.0   

   5565  5566  5567  5568  5569  5570  5571  5572  5573  
0   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  
1   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  
2   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  
3   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  
4   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0  

[5 rows x 5574 columns]


In [21]:
#se identificaron las columnas categóricas y aplicado One-Hot Encoding a las variables genus y species. La columna accession fue excluida, y la columna phenotype fue manejada como una etiqueta en pasos previos.

In [22]:
#La razón por la que el número de columnas ha crecido tanto es porque el proceso de One-Hot Encoding transforma cada categoría en una columna separada (binaria), lo que genera un aumento significativo en el número de columnas.